In [1]:
import sys
import glob
import torch
sys.path.append('../')
import os
import numpy as np
from transformers import *
from kaiser.src import utils
from kaiser.src import dataio
from kaiser import target_identifier
from kaiser import inference
from kaiser.src.modeling import BertForJointShallowSemanticParsing
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm, trange


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

I1117 17:15:44.863185 140388328597248 file_utils.py:39] PyTorch version 0.4.1 available.
I1117 17:15:45.022875 140388328597248 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Using TensorFlow backend.


### Korean FrameNet ###
	# contact: hahmyg@kaist, hahmyg@gmail.com #



In [5]:
text = '안녕 나는 <tgt> 사랑해 </tgt>'

bert_io = utils.for_BERT(mode='predict')      

In [7]:
t = bert_io.bert_tokenizer(text)
print(t)
print(bert_io.tokenizer.convert_tokens_to_ids(t[1]))

(['안녕', '나는', '<tgt>', '사랑해', '</tgt>'], ['[CLS]', '안', '##녕', '나는', '<tgt>', '사', '##랑', '##해', '</tgt>', '[SEP]'], [1, 3, 4, 5, 8])
[101, 9521, 118741, 100585, 1, 9405, 62200, 14523, 2, 102]


In [8]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')
input_ids = torch.tensor(bert_io.tokenizer.convert_tokens_to_ids(t[1])).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

I1117 17:26:04.724481 140388328597248 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /home/hahmyg/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
I1117 17:26:04.728356 140388328597248 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_

In [9]:
print(last_hidden_states)

tensor([[[ 3.0073e-01, -1.4202e-01,  3.8447e-01,  ...,  6.1753e-01,
          -1.3053e-01,  3.0373e-01],
         [ 1.7382e-01, -4.1211e-01,  3.5874e-01,  ...,  1.1530e+00,
           9.4785e-02,  1.2455e-01],
         [ 2.9706e-01, -4.0844e-01,  7.3426e-01,  ...,  8.9539e-01,
          -1.1355e-01,  1.9625e-01],
         ...,
         [ 5.0557e-01,  1.1449e-01,  5.6316e-01,  ...,  7.3965e-01,
          -4.5839e-01,  6.3396e-01],
         [ 6.6071e-01, -2.9220e-01,  7.8144e-01,  ...,  6.6861e-01,
          -6.5892e-01,  5.4593e-01],
         [ 5.3778e-01, -3.0699e-01,  8.5603e-01,  ...,  7.9521e-01,
          -2.0699e-01,  3.8107e-01]]], grad_fn=<AddcmulBackward>)
